<a href="https://colab.research.google.com/github/AjayBora002/movie_recommendation/blob/main/Copy_of_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Import Modules


In [32]:
import pandas as pd
import numpy as np
import re
import nltk

In [33]:
movies=pd.read_csv("movies.csv")

In [34]:


def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title.lower()).strip()


In [35]:
movies["clean_title"]=movies["title"].apply(clean_title)# this will access a new clean title column


2. FINDING SIMILARITIES BETWEEN THE WORDS AND WITH THE HELP OF IT GIVING THE SIMILAR TYPE OF MOVIE TO THE USER

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer# converting text data into numerical data and finding similarities , this will do it
vectorizer = TfidfVectorizer(ngram_range=(1,2))# this ngram thing will help in finding instead of single single word it will match two consecutive words like toy story together
tfidf = vectorizer.fit_transform(movies["clean_title"])

3.finding similarity and creating search function

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)  # clean the input title
    query_vec = vectorizer.transform([title])  # vectorize it
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results


4.CREATING AN INTERACTIVE BOX

In [38]:
import ipywidgets as widgets
from IPython.display import display  # function used to show diiff things as output on notebook


movie_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)


# this whole thing will create a box to enter movie title , but wont show anything as output
  #NOW MAKING AN OUTPUT WIDGET
movie_list = widgets.Output()

def on_type(data):     # this fun will be called whenever we type something in the box
  with movie_list:
    movie_list.clear_output()

    title=data["new"]
    if len(title)>5:
      display(search(title))

movie_input.observe(on_type, names='value')  # there are diff events of the widgets whenever we input a movie name it is going to call on type and gives value event
display(movie_input, movie_list)




Text(value='Toy Story', description='Movie Title :')

Output()

In [39]:
ratings=pd.read_csv("ratings.csv")

In [40]:
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


5.FINDING SIMILAR USER RECOMMENDATION

In [41]:
movie_id=1

In [42]:
similar_users = ratings[(ratings["movieId"] == movie_id)# this is for finding anyone who watched same movie
& (ratings["rating"] > 4)]["userId"].unique()# this is for anyone who watched the movie and gave the rating 5

similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
# this will show the movies that are rated by similar users and find their used ids

In [43]:
similar_user_recs

,movieId
5101,1
5105,34
5111,110
5114,150
5127,260
...,...
534921,58559
534922,58998
534925,60069
534932,68954


In [44]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users) # this counts the no of 5 rates is given to a movie and then div by len shows recommending percent like top one has 100%

# we are doing this to find per of similar people who watched the movie
similar_user_recs = similar_user_recs[similar_user_recs > .10]  # shows movies having per greater than 10 %

In [45]:
similar_user_recs

,count
movieId,
1,1.000000
318,0.453012
260,0.383133
356,0.337349
593,0.332530
...,...
551,0.106024
1207,0.103614
2396,0.103614


6.now problem is that most of the people like movies like toy story and avengers , what we need is how much movies are liked by users are we also like this will give more accuracy on recommendation

In [46]:
all_users=ratings[(ratings)["movieId"].isin(similar_user_recs.index) & (ratings["rating"]>4)]

In [47]:
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())# this will give us percentage of all the users who liked the movies

In [48]:
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)# this concatenates the data of how much similar people to us liked and how much avg person liked it
rec_percentages.columns=["similar","all"]

In [49]:
rec_percentages

,similar,all
movieId,,
1,1.000000,0.122708
318,0.453012,0.343584
260,0.383133,0.214075
356,0.337349,0.220284
593,0.332530,0.225310
...,...,...
551,0.106024,0.044057
1207,0.103614,0.047309
2396,0.103614,0.051449


7.NOW WE WANT MOVIES THAT HAS A BIG DIFF BET THESE TWO COLUMN

In [50]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages=rec_percentages.sort_values("score",ascending=False)   # using pandas sort method to make an order of the score the higher the score more good will be the recommendation
rec_percentages

,similar,all,score
movieId,,,
1,1.000000,0.122708,8.149398
3114,0.269880,0.053223,5.070736
78499,0.154217,0.033116,4.656799
2355,0.113253,0.025724,4.402548
588,0.226506,0.066824,3.389572
...,...,...,...
7361,0.122892,0.104967,1.170759
2329,0.106024,0.090775,1.167992
2858,0.185542,0.163513,1.134726


8.NOW TAKING TOP 10 RATINGS AND MERGE IT WITH MOVIES DATA

In [51]:
rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.122708,8.149398,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
3021,0.269880,0.053223,5.070736,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,toy story 2 1999
14813,0.154217,0.033116,4.656799,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,toy story 3 2010
2264,0.113253,0.025724,4.402548,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,bugs life a 1998
580,0.226506,0.066824,3.389572,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,aladdin 1992
4780,0.221687,0.065642,3.377228,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,monsters inc 2001
11361,0.120482,0.038439,3.134384,50872,Ratatouille (2007),Animation|Children|Drama,ratatouille 2007
587,0.197590,0.064163,3.079496,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,beauty and the beast 1991
1047,0.156627,0.052632,2.975904,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,willy wonka the chocolate factory 1971
2025,0.110843,0.037552,2.951750,2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure|Fantasy,indiana jones and the temple of doom 1984


9.PUTTING ALL OF THIS INTO A FUNCTION

In [52]:
def find_similar_movies(movie_id):

    similar_users = ratings[(ratings["movieId"] == movie_id)
    & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) # this is finding recommendation similar to us
    & (ratings["rating"] > 4)]["movieId"]

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)# adjusting the per of recommendation
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]# this is finding common recommendation among all of the users
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)# concatinating
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]# generating score

    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]  #sorting and returning our merge data

10.CREATING AN INTERACTIVE RECOMMENDATION WIDGET

In [53]:
movie_name_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)

recommendation_list=widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()# removes old output
    title=data["new"]
    if len(title)>5:
      results=search(title)
      movie_id=results.iloc[0]["movieId"]
      display(find_similar_movies(movie_id))

movie_name_input.observe(on_type,names="value")
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title :')

Output()